# Initializations

In [ ]:
!gdown 'https://drive.google.com/uc?id=1TD8Uk8WVVTs1JGh1GhHHz827UGv5RTNx&export=download&authuser=0&confirm=t&uuid=6e5b24cf-991f-4b7d-a1af-80ef93cdd15e';
!git clone https://github.com/nadimkanazi/e2e-pianoform.git

Downloading...
From: https://drive.google.com/uc?id=1TD8Uk8WVVTs1JGh1GhHHz827UGv5RTNx&export=download&authuser=0&confirm=t&uuid=6e5b24cf-991f-4b7d-a1af-80ef93cdd15e
To: /content/grandstaff.tgz
100% 952M/952M [00:27<00:00, 34.8MB/s]
Cloning into 'e2e-pianoform'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 57 (delta 22), reused 32 (delta 10), pack-reused 0
Receiving objects: 100% (57/57), 275.42 KiB | 3.24 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [ ]:
#unzip the tgz raw data file.
!mkdir -p Data/grandstaff_dataset
!tar -xzvf "grandstaff.tgz" -C "Data/grandstaff_dataset"
!pip install evaluate
!pip install transformers

Streaming output truncated to the last 5000 lines.
scarlatti-d/keyboard-sonatas/L304K470/min3_down_m-120-125.jpg.gm
scarlatti-d/keyboard-sonatas/L304K470/maj2_up_m-12-15.bekrn
scarlatti-d/keyboard-sonatas/L304K470/original_m-0-3.bekrn
scarlatti-d/keyboard-sonatas/L304K470/maj2_down_m-60-64.krn
scarlatti-d/keyboard-sonatas/L304K470/original_m-36-39.jpg.gm
scarlatti-d/keyboard-sonatas/L304K470/maj2_up_m-99-102.jpg.gm
scarlatti-d/keyboard-sonatas/L304K470/maj3_down_m-69-72.jpg.gm
scarlatti-d/keyboard-sonatas/L304K470/maj2_up_m-117-120_distorted.jpg
scarlatti-d/keyboard-sonatas/L304K470/min3_up_m-36-39.jpg.gm
scarlatti-d/keyboard-sonatas/L304K470/original_m-60-63.jpg
scarlatti-d/keyboard-sonatas/L304K470/maj3_down_m-93-96.bekrn
scarlatti-d/keyboard-sonatas/L304K470/maj2_down_m-28-32.jpg
scarlatti-d/keyboard-sonatas/L304K470/min3_up_m-72-75.krn
scarlatti-d/keyboard-sonatas/L304K470/min3_up_m-3-6.bekrn
scarlatti-d/keyboard-sonatas/L304K470/original_m-69-72.bekrn
scarlatti-d/keyboard-sonatas/

In [ ]:
!mv 'e2e-pianoform/train.txt' 'Data/grandstaff_dataset/train.txt'
!mv 'e2e-pianoform/test.txt' 'Data/grandstaff_dataset/test.txt'
!mv 'e2e-pianoform/val.txt' 'Data/grandstaff_dataset/val.txt'

### Create Dataset datasets

In [ ]:
"""Note: In this section, we borrow/adapt code from https://github.com/multiscore/e2e-pianoform, the original source code from the authors that designed and created the raw dataset"""

'Note: In this section, we borrow/adapt code from https://github.com/multiscore/e2e-pianoform, the original source code from the authors that designed and created the raw dataset'

In [ ]:
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
import sys
import torch
sys.path.append("/content/e2e-pianoform")
from torch.utils.data import DataLoader
from data import load_dataset, batch_preparation_ctc, check_and_retrieveVocabulary, PoliphonicDataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_path = "Data/grandstaff_dataset/train.txt"
val_path = "Data/grandstaff_dataset/val.txt"
test_path = "Data/grandstaff_dataset/test.txt"

train_dataset = PoliphonicDataset(partition_file=train_path)
val_dataset = PoliphonicDataset(partition_file=val_path)
test_dataset = PoliphonicDataset(partition_file=test_path)

print(len(train_dataset))


Output()

Output()

Output()

32329


In [ ]:
def make_vocabulary(YSequences, pathToSave, nameOfVoc):
    vocabulary = set()
    for samples in YSequences:
        for element in samples:
                vocabulary.update(element)

    print(len(vocabulary))
    #Vocabulary created
    w2i = {symbol:idx+1 for idx,symbol in enumerate(vocabulary)}
    i2w = {idx+1:symbol for idx,symbol in enumerate(vocabulary)}

    w2i['<pad>'] = 0
    i2w[0] = '<pad>'

    return w2i, i2w

In [ ]:
import numpy as np
#for i in val_dataset:
#    print(i)
#print(train_dataset.get_gt())
w2i, i2w = make_vocabulary([train_dataset.get_gt(), val_dataset.get_gt(), test_dataset.get_gt()], "vocab", f"testy")
print(w2i)
print(i2w)

187
{'q': 1, 'LL': 2, '[<': 3, '[>': 4, '*M12/8': 5, '*k[f#c#g#d#]': 6, 'i': 7, 'BBB': 8, 'LK>': 9, '##': 10, '*M4/4': 11, '*M8/2': 12, '36': 13, '32': 14, '*M12/16': 15, 'CC': 16, '*met(c)': 17, 'JJJk': 18, 'Jkk': 19, 'DD': 20, '*M12/32': 21, 'EEE': 22, '30': 23, 'L<': 24, '*v': 25, 'GGG': 26, '*k[b-]': 27, '**ekern_1.0': 28, '..': 29, '*k[]': 30, 'r': 31, 'B': 32, 'aaaa': 33, 'LL<': 34, '*k[f#c#g#d#a#]': 35, '=!|:': 36, '28': 37, ']': 38, '34': 39, 'Jk': 40, '.': 41, 'x': 42, '=': 43, 'J': 44, '=:|!|:': 45, 'JJJJ': 46, '8': 47, '*M17/16': 48, 'bbb': 49, 'JJk': 50, '*k[f#c#]': 51, 'Z': 52, '<t>': 53, 'L': 54, '*M9/8': 55, '=|!': 56, '==:|!': 57, 'ddd': 58, 'A': 59, 'eeee': 60, '*staff2': 61, '4': 62, '1': 63, '*met(c|)': 64, '48': 65, '*k[b-e-a-d-g-]': 66, '#': 67, 'j': 68, 'aaa': 69, 'KK': 70, '*k[b-e-a-d-g-c-]': 71, '*k[f#c#g#]': 72, 'a': 73, 'LLLL>': 74, 'g': 75, '64': 76, '*k[f#c#g#d#a#e#]': 77, '*k[b-e-a-d-g-c-f-]': 78, 'JJ': 79, 'Lk': 80, 'AAA': 81, '=-': 82, '<': 83, '*M2/2': 8

In [ ]:
train_dataset.set_dictionaries(w2i, i2w)
val_dataset.set_dictionaries(w2i, i2w)
test_dataset.set_dictionaries(w2i, i2w)

train_dataloader = DataLoader(train_dataset, batch_size=32, num_workers=1, collate_fn=batch_preparation_ctc)
val_dataloader = DataLoader(val_dataset, batch_size=32, num_workers=1, collate_fn=batch_preparation_ctc)
test_dataloader = DataLoader(test_dataset, batch_size=32, num_workers=1, collate_fn=batch_preparation_ctc)

In [ ]:
a = next(iter(train_dataloader))
print(a[0].size())
print(a[1].size())
#for i in train_dataloader:
#    print(i)

torch.Size([32, 1, 1164, 256])
torch.Size([32, 504])


In [ ]:
train_dataset[0][0].shape, train_dataset[0][1].shape, train_dataset[0][2], train_dataset[0][3]

(torch.Size([1, 670, 256]), torch.Size([279]), 1312, 279)

# Model

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from transformers import BertConfig, BertModel, BertTokenizer
import torch.backends.cudnn as cudnn
from sklearn.metrics import accuracy_score
import time
import copy

class OMRModel(nn.Module):
    def __init__(self, vocab_size, gru_hidden=256, gru_layers=4, max_sequence_length=1716):
        super(OMRModel, self).__init__()
        self.vocab_size=vocab_size

        densenet = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
        densenet.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        children = list(densenet.children())
        self.image_encoder = nn.Sequential(*children[:-1])
        self.image_encoder = self.image_encoder[0][:7]
        #output of this has size [1, 512, 64, 32]

        # sequence encoder
        self.gru1 = nn.GRU(2048, gru_hidden, num_layers=gru_layers//2, batch_first=True, bidirectional=True)
        self.gru2 = nn.GRU(gru_hidden*2, gru_hidden, num_layers=gru_layers//2, batch_first=True, bidirectional=True)



        # sequence decoder
        self.embedding = nn.Embedding(vocab_size, gru_hidden)
        self.attn = nn.MultiheadAttention(embed_dim=gru_hidden, num_heads=4)
        self.decoder = nn.GRU(gru_hidden, gru_hidden, num_layers=gru_layers//2, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(gru_hidden*4, vocab_size)
        self.clf = nn.Linear(vocab_size, 1)

        for param in self.image_encoder.parameters():
            param.requires_grad = False

    def forward(self, imgs, L, T):
        batch_size = imgs.size()[0]
        #L is input sequence length and T is target sequence length
        #resize image
        resize_transform = transforms.Resize((512, 256))
        imgs = resize_transform(imgs)

        #encode image
        image_features = self.image_encoder(imgs)

        #flatten image into column wise elements to match sheet music structure
        image_fearues = image_features.permute(0, 2, 1, 3).contiguous().view(1, 32, -1)
        image_features = torch.cat([image_features[:, i, :] for i in range(32)], dim=1)

        #encode sequence with residual connection

        o1, h1 = self.gru1(image_features.transpose(1,2))
        o2, h2 = self.gru2(o1)


        # Initialize hidden state of decoder
        hidden = h2

        # Decode sequence
        embedded = self.embedding(torch.zeros(imgs.size(0), 1, dtype=torch.long, device=imgs.device))  # Start token

        outputs = torch.zeros((batch_size, max(T), self.vocab_size))
        for i in range(len(T)):
            for t in range(T[i]):
                output, hidden = self.decoder(embedded, hidden)
                output = output.unsqueeze(1)  # Add time dimension
                attn_output, _ = self.attn(hidden, h2, h2)  # Compute attention

                attn_output = torch.reshape(attn_output, (1,-1))
                output = torch.reshape(output, (1,-1))
                concat_output = torch.cat((output, attn_output), dim=-1)  # Concatenate output and attention
                concat_output = torch.reshape(concat_output, (batch_size, 1024))
                output = self.fc(concat_output)  # Project to vocabulary size
                outputs[:,t,:] = output
                embedded = self.embedding(output.argmax(dim=-1))  # Use predicted token as next input
                embedded = embedded.unsqueeze(1)
        outputs = outputs.to(device)
        predict = self.clf(outputs)
        return predict


# Trainig Loop

In [ ]:
# Training loop
def train_model(model, criterion, optimizer, dataloaders, device, train_size, val_size, num_epochs=10, scheduler=None):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # The name of the directory to save all the checkpoints
    timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
    model_dir = os.path.join('models', timestr)

    # Keep track of the checkpoint with the smallest test loss and save in model_path
    model_path = None

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        i = 0
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for images, labels, L, T in dataloaders[phase]:
                images = images.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    print(i)
                    i+=1
                    outputs = model.forward(images, L, T)
                    outputs = outputs.squeeze().unsqueeze(0)
                    labels = labels.squeeze().unsqueeze(0)
                    print(f'L: {L}')
                    print(f'T: {T}')
                    print(f'outputs: {outputs.size()}')
                    print(f'labels: {labels.size()}')
                    _, preds = torch.max(outputs, 0)
                    loss = criterion(outputs.float(), labels.float())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # Statistics
                running_loss += loss.item() * images.size(0)
                running_corrects += torch.sum(preds == labels.data)


            if phase == 'train' and scheduler is not None:
                scheduler.step()
            epoch_loss = running_loss / train_size if phase == 'train' else running_loss / val_size
            epoch_acc = running_corrects.double() / train_size if phase == 'train' else running_corrects.double() / val_size

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            model_folder_name = f'epoch_{epoch:04d}_loss_{epoch_loss:.8f}'

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                if not os.path.exists(os.path.join(model_dir, model_folder_name)):
                    os.makedirs(os.path.join(model_dir, model_folder_name))
                torch.save(model.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics.pth'))
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        #print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Eval fcnt

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, tokens, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            tokens = tokens.to(device)
            outputs = model(images, tokens)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return accuracy_score(all_labels, all_preds)

# Misc

In [ ]:
import pdb
import evaluate
import warnings
from torch.optim import Adam

# Suppress a few warnings for printing purposes
warnings.filterwarnings("ignore", category=UserWarning, message="The parameter 'pretrained' is deprecated since 0.13.*")
warnings.filterwarnings("ignore", category=UserWarning, message="Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13.*")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="os.fork() was called.*")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
dataloaders = {'train': train_dataloader,'val': val_dataloader}
train_size = len(train_dataset)
val_size = len(val_dataset)
model = OMRModel(len(w2i), 256, 2, 1716)
#def __init__(self, vocab_size, rnn_dim, rnn_layers, max_predict_length):
#model = Model(len(w2i), 256, 2, 1716)
#'vocab_size', 'rnn_dim', 'rnn_layers', and 'max_predict_length'
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)
model.to(device)

tr_model = train_model(model, criterion, optimizer, dataloaders, device, train_size, val_size)

# evaluate model:
#L is (image.shape[2] // 8) * (image.shape[1] // 16)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch 0/9
----------


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0
L: [1312, 2016, 1664, 864, 1440, 640, 1088, 1536, 1024, 1760, 1472, 960, 1760, 1152, 1152, 1248, 2304, 1280, 1568, 1184, 1152, 1344, 1312, 1472, 1184, 1536, 1216, 1920, 1184, 1760, 1088, 1152]
T: [279, 407, 362, 249, 245, 81, 262, 328, 428, 342, 292, 227, 316, 500, 207, 222, 504, 257, 423, 223, 290, 413, 259, 307, 369, 331, 347, 404, 333, 369, 286, 218]
outputs: torch.Size([1, 32, 504])
labels: torch.Size([1, 32, 504])
